# Eco Score Pipeline for beautofuel

In [ ]:
%load_ext autoreload
%autoreload 2

from influxdb import DataFrameClient

from lib.utils.constants import INFLUXDB_HOST, INFLUXDB_PORT, INFLUXDB_USER, INFLUXDB_PASSWORD, INFLUXDB_DB

## Pipeline configuration

In [ ]:
# Database client
grafanadb = DataFrameClient(
    host=INFLUXDB_HOST,
    port=INFLUXDB_PORT,
    database=INFLUXDB_DB,
    username=INFLUXDB_USER,
    password=INFLUXDB_PASSWORD
)

user = "zlatka"

## Helper functions

In [ ]:
def get_query_result_value(result, field):
    return result['tracks'][field][0]

def get_query_end(phase):
    return "FROM \"tracks\" WHERE (\"phase\"='{}' AND \"user\"='{}')".format(phase, user)

## Setup Queries

In [ ]:
# Consumption queries
stdddev_consumption_query = "SELECT stddev(\"consumption\") {};".format(get_query_end(1))
mean_consumption_query = "SELECT mean(\"consumption\") {};".format(get_query_end(1))
min_consumption_query = "SELECT min(\"consumption\") {};".format(get_query_end(1))
max_consumption_query = "SELECT max(\"consumption\") {};".format(get_query_end(1))

## Execute Queries

In [ ]:
# Execute consumption queries
stddev_consumption = get_query_result_value(grafanadb.query(stdddev_consumption_query), 'stddev')
mean_consumption = get_query_result_value(grafanadb.query(mean_consumption_query), 'mean')
min_consumption = get_query_result_value(grafanadb.query(min_consumption_query), 'min')
max_consumption = get_query_result_value(grafanadb.query(max_consumption_query), 'max')

In [ ]:
# Print consumption queries results
print("consumption_100_std:", stddev_consumption)
print("consumption_100_mean:", mean_consumption)
print("consumption_100_min:", min_consumption)
print("consumption_100_max:", max_consumption)

## Eco-score Calculation

In [ ]:
tracks_2_query = "SELECT \"consumption\" {};".format(get_query_end(2))

tracks_2 = grafanadb.query(tracks_2_query)

for _, track_row in tracks_2['tracks'].iterrows():
    track_2_consumption = track_row['consumption']

    part_50 = None
    part_30 = None
    part_20 = None
    part_10 = None

    lower_consumption_limit = mean_consumption - stddev_consumption
    upper_consumption_limit = mean_consumption + stddev_consumption

    if track_2_consumption >= upper_consumption_limit:
        part_50 = 0
    elif track_2_consumption <= lower_consumption_limit:
        part_50 = 100
    else:
        track_2_consumption = track_2_consumption - lower_consumption_limit
        upper_consumption_limit = upper_consumption_limit - lower_consumption_limit
        part_50 = int((track_2_consumption / upper_consumption_limit) * 100)

    print("50%:", part_50)

    if track_2_consumption < min_consumption:
        part_30 = 100
    else:
        part_30 = 0

    print("30%:", part_30)

    if track_2_consumption < mean_consumption:
        part_20 = 100
    else:
        part_20 = 0

    print("20%:", part_20)

    if track_2_consumption < max_consumption:
        part_10 = 100
    else:
        part_10 = 0

    print("10%:", part_10)

    eco_score = part_50 * 0.5 + part_30 * 0.3 + part_20 * 0.2 + part_10 * 0.1

    print("eco_score:", int(eco_score))

    print()